In [ ]:
# === Imports === #
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import yaml
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Load Configuration === #
def load_config(path):
    """Load configuration from YAML file."""
    with open(path, "r") as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# === Retrieve Account Information === #
account_info = mt5.account_info()
if account_info is None:
    print("Failed to retrieve account information.")
    mt5.shutdown()
    quit()

# === Extract Balance, Equity, and Floating Loss === #
balance = account_info.balance
equity = account_info.equity
floating_loss = account_info.profit

# === Display Results === #
print("=== Account Information ===")
print(f"Balance: {balance:.2f} USD")
print(f"Equity: {equity:.2f} USD")
print(f"Floating Loss: {floating_loss:.2f} USD")

# === Fetch and Process Running Trades === #
fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
df = fetcher.get_sheet_as_dataframe("Forex")
positions = mt5.positions_get()
df_positions = pd.DataFrame([pos._asdict() for pos in positions]) if positions else pd.DataFrame()

if df_positions.empty:
    print("No running trades found. Error:", mt5.last_error())
else:
    # Filter and rename columns
    df_positions = df_positions[["type", "volume", "price_open", "price_current", "sl", "tp", "profit", "symbol"]]
    df_positions["type"] = df_positions["type"].replace({0: "Buy", 1: "Sell"})
    df_positions.rename(columns={
        "symbol": "Symbol",
        "type": "Action",
        "volume": "Lot",
        "price_open": "Price",
        "price_current": "Price Current",
        "sl": "SL",
        "tp": "TP",
        "profit": "Profit"
    }, inplace=True)

    # Aggregate by Symbol and Action
    def aggregate_group(group):
        symbol = group["Symbol"].iloc[0]
        action = group["Action"].iloc[0]
        lot = group["Lot"].sum()
        warnings = []

        # === Lot mismatch check === #
        df_lot_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[5]].sum()
        if not pd.isna(df_lot_value) and round(df_lot_value, 2) > round(lot, 2):
            warnings.append(f"Lot Mismatch: Expected {round(df_lot_value, 2)}, Found {round(lot, 2)}")

        # === SL/TP mismatch check === #
        df_sl_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[7]].sum()
        sl_mode = group["SL"].mode().iloc[0] if not group["SL"].mode().empty else np.nan
        if group["SL"].nunique() > 1 or (df_sl_value != 0 and round(df_sl_value, 2) != round(sl_mode, 2)):
            warnings.append("SL Mismatch")

        df_tp_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[8]].sum()
        tp_mode = group["TP"].mode().iloc[0] if not group["TP"].mode().empty else np.nan
        if group["TP"].nunique() > 1 or round(df_tp_value, 2) != round(tp_mode, 2):
            warnings.append("TP Mismatch")

        # Combine warnings
        warning_message = "; ".join(warnings) if warnings else np.nan

        return pd.Series({
            "Lot": lot,
            "Price": (group["Price"] * group["Lot"]).sum() / group["Lot"].sum(),
            "Price Current": group["Price Current"].mean(),
            "SL": group["SL"].mode().iloc[0] if not group["SL"].empty else np.nan,
            "TP": group["TP"].mode().iloc[0] if not group["TP"].empty else np.nan,
            "Profit": group["Profit"].sum(),
            "Warning": warning_message
        })

    df_positions = (
        df_positions.groupby(["Symbol", "Action"], as_index=False)
        .apply(lambda group: aggregate_group(group))
        .reset_index(drop=True)
    )

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Calculate Coefficients === #
df_positions["Coeff"] = np.abs(df_positions["Profit"] / (df_positions["Price Current"] - df_positions["Price"])) / (df_positions["Lot"] * 100)
df_positions["Pips"] = np.abs(df_positions["TP"] - df_positions["SL"])

# Merging the Type column based on Symbol
df_type = df[[df.columns[0], df.columns[1]]].drop_duplicates()
df_type.columns = ["Symbol", "Type"]
df_positions = pd.merge(df_positions, df_type, on="Symbol", how="left")

# Handling missing types (optional)
df_positions['Type'].fillna("Unknown", inplace=True)

df_positions

# === Update Coefficients in Config (Exclude NaN) === #
df_avg_coeff = df_positions.groupby("Symbol")["Coeff"].mean().reset_index()
differences = {}

for _, row in df_avg_coeff.iterrows():
    symbol = row["Symbol"]
    avg_coeff = round(row["Coeff"], 5)
    
    # Only update if coefficient is not NaN
    if not pd.isna(avg_coeff):
        if symbol in config.get("symbol_coefficients", {}):
            old_coeff = config["symbol_coefficients"].get(symbol, avg_coeff)
            if old_coeff != avg_coeff:
                differences[symbol] = {"Old": old_coeff, "New": avg_coeff}
        
        # Add or update the coefficient
        config["symbol_coefficients"][symbol] = avg_coeff

# === Save Updated Configuration === #
with open(CONFIG_PATH, "w") as file:
    yaml.safe_dump(config, file)

# === Display Changed Coefficients === #
print("Changed symbol_coefficients:")
for symbol, change in differences.items():
    print(f"{symbol}: {change['Old']} -> {change['New']}")

# Creating new rows for Balance, Equity, and Floating Loss
df_balance = pd.DataFrame([
    {"Symbol": "Balance", "Price": balance},
    {"Symbol": "Equity", "Price": equity},
    {"Symbol": "Floating Loss", "Price": floating_loss}
])

# Inserting the new rows at the top of the DataFrame
df_final = pd.concat([df_balance, df_positions], ignore_index=True)

# === Uploade Updated Dataframe === #
uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
uploader.upload_dataframe(df_final, "Forex Summary", replace=True)

# === Display Final Merged DataFrame === #
print("Final Merged DataFrame:")
df_final

In [ ]:
import time

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()
    
# Load DataFrame from Google Sheets
df = fetcher.get_sheet_as_dataframe("Forex")
df.columns = df.iloc[5]
df = df.iloc[6:-3].reset_index(drop=True)

# Initialize counters for success and failure
success_count = 0
failure_count = 0

# Iterate through each unique symbol and action
symbol_list = df["Symbol"].unique()
for symbol in symbol_list:
#for symbol in ["ADAUSD"]:
    for action in ["Buy", "Sell"]:
        # Filter rows for the current symbol and action
        df_filtered = df[(df["Symbol"] == symbol) & (df["Action"] == action)]
        
        if not df_filtered.empty:
            print(f"Symbol: {symbol}")
            print(f"Action: {action}")

            tp_value = round(df_filtered['TP'].values[0], 5)
            sl_value = round(df_filtered['SL'].values[0], 5)

            # === Fetch Open Positions for the Specified Symbol ===
            positions = mt5.positions_get(symbol=symbol)
            if positions is None or len(positions) == 0:
                print(f"No open positions found for {symbol}.")
                continue

            # === Modify TP/SL for All Trades based on Action ===
            for pos in positions:
                action_type = "Buy" if pos.type == 0 else "Sell"
                
                # Ensure it only modifies the correct action type (Buy or Sell)
                if action_type == action:
                    order_id = pos.ticket
                    current_tp = pos.tp
                    current_sl = pos.sl

                    # Only modify if the new TP/SL values differ from current
                    if current_tp == tp_value and current_sl == sl_value:
                        print(f"✅ No changes for {action_type} position {order_id}. TP/SL already set.")
                        continue

                    # Modify TP/SL using position ID
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": order_id,
                        "sl": sl_value,
                        "tp": tp_value
                    }

                    # Send modification request
                    result = mt5.order_send(request)
                    time.sleep(1)  # Delay of 2 seconds between each request

                    # Check if result is None (Error)
                    if result is None:
                        print(f"⚠️ Failed to modify {action_type} position {order_id}. Error: Request returned None.")
                        failure_count += 1
                        continue

                    # Check if the modification was successful
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"{action_type} position {order_id} modified successfully.")
                        print(f"Before: TP={current_tp}, SL={current_sl}")
                        print(f"After:  TP={tp_value}, SL={sl_value}\n")
                        success_count += 1
                    else:
                        print(f"⚠️ Failed to modify {action_type} position {order_id}. Error: {result.comment}")
                        failure_count += 1
                else:
                    print(f"⚠️ Skipping {action_type} position {pos.ticket} - does not match action {action}.")

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Summary Report ===
print("\n=== TP/SL Modification Summary ===")
print(f"Total Successful Modifications: {success_count}")
print(f"Total Failed Modifications: {failure_count}")

✅ Successfully retrieved data from 'Forex' as a DataFrame.
Symbol: AAPL.NAS
Action: Buy
⚠️ Failed to modify Buy position 3297549570. Error: Market closed
Symbol: AAVEUSD
Action: Buy
⚠️ Failed to modify Buy position 3302602638. Error: Request returned None.
Symbol: ABBV.NYSE
Action: Buy
⚠️ Failed to modify Buy position 3298446412. Error: Market closed
Symbol: ABNB.NAS
Action: Buy
⚠️ Failed to modify Buy position 3304860529. Error: Request returned None.
Symbol: ABT.NYSE
Action: Buy
⚠️ Failed to modify Buy position 3302221724. Error: Market closed
Symbol: ACN.NYSE
Action: Buy
⚠️ Failed to modify Buy position 3304837038. Error: Market closed
Symbol: ADAUSD
Action: Buy
✅ No changes for Buy position 3304100487. TP/SL already set.
Symbol: ADBE.NAS
Action: Buy
No open positions found for ADBE.NAS.
Symbol: ADS.XE
Action: Buy
⚠️ Failed to modify Buy position 3305209594. Error: No changes
Symbol: AI.EPA
Action: Buy
⚠️ Failed to modify Buy position 3297913700. Error: No changes
Symbol: AIR.EPA
Ac

In [ ]:
import time
import MetaTrader5 as mt5
import pandas as pd

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# Load DataFrame from Google Sheets
df = fetcher.get_sheet_as_dataframe("Forex")
df.columns = df.iloc[5]
df = df.iloc[6:-3].reset_index(drop=True)

# Initialize counters for success and failure
success_count = 0
failure_count = 0

# Fetch unique symbols from running trades in MT5
positions = mt5.positions_get()
if positions is None or len(positions) == 0:
    print("No running trades found. Exiting.")
    mt5.shutdown()
    quit()

# Get unique symbols from MT5 positions
symbol_list = sorted(list(set(pos.symbol for pos in positions)))

# Convert positions to DataFrame for easy filtering
df_positions = pd.DataFrame([pos._asdict() for pos in positions])

# Iterate through each unique symbol from MT5 positions
for symbol in symbol_list:
    for action in ["Buy", "Sell"]:
        # Filter positions for the current symbol and action
        action_type = 0 if action == "Buy" else 1
        filtered_positions = df_positions[(df_positions['symbol'] == symbol) & (df_positions['type'] == action_type)]
        
        if filtered_positions.empty:
            print(f"Symbol: {symbol}\nAction: {action}\n⚠️ No {action} positions running. Skipping.")
            continue

        # Filter rows for the current symbol and action from Google Sheets
        df_filtered = df[(df["Symbol"] == symbol) & (df["Action"] == action)]
        
        if df_filtered.empty:
            print(f"Symbol: {symbol}\nAction: {action}\n⚠️ No matching TP/SL settings in Google Sheets.")
            continue
        
        tp_value = round(df_filtered['TP'].values[0], 5)
        sl_value = round(df_filtered['SL'].values[0], 5)

        # Iterate over filtered positions for modification
        for _, pos in filtered_positions.iterrows():
            order_id = pos['ticket']
            current_tp = pos['tp']
            current_sl = pos['sl']

            # Only modify if the new TP/SL values differ from current
            if current_tp == tp_value and current_sl == sl_value:
                print(f"Symbol: {symbol}\nAction: {action}\n✅ No changes for {action} position {order_id}. TP/SL already set.")
                continue

            # Modify TP/SL using position ID
            request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": order_id,
                "sl": sl_value,
                "tp": tp_value
            }

            # Send modification request
            result = mt5.order_send(request)
            #time.sleep(0.5)  # Delay of 0.5 second between each request

            # Check if result is None (Error)
            if result is None:
                print(f"Symbol: {symbol}\nAction: {action}\n⚠️ Failed to modify {action} position {order_id}. Error: Request returned None.")
                failure_count += 1
                continue

            # Check if the modification was successful
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"Symbol: {symbol}\nAction: {action}\n✅ {action} position {order_id} modified successfully.")
                print(f"Before: TP={current_tp}, SL={current_sl}")
                print(f"After:  TP={tp_value}, SL={sl_value}\n")
                success_count += 1
            else:
                print(f"Symbol: {symbol}\nAction: {action}\n⚠️ Failed to modify {action} position {order_id}. Error: {result.comment}")
                failure_count += 1

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Summary Report ===
print("\n=== TP/SL Modification Summary ===")
print(f"Total Successful Modifications: {success_count}")
print(f"Total Failed Modifications: {failure_count}")

✅ Successfully retrieved data from 'Forex' as a DataFrame.
Symbol: AAPL.NAS
Action: Buy
⚠️ Failed to modify Buy position 3297549570. Error: AutoTrading disabled by client
Symbol: AAPL.NAS
Action: Sell
⚠️ No Sell positions running. Skipping.
Symbol: AAVEUSD
Action: Buy
⚠️ Failed to modify Buy position 3302602638. Error: Request returned None.
Symbol: AAVEUSD
Action: Sell
⚠️ No Sell positions running. Skipping.
Symbol: ABBV.NYSE
Action: Buy
⚠️ Failed to modify Buy position 3298446412. Error: AutoTrading disabled by client
Symbol: ABBV.NYSE
Action: Sell
⚠️ No Sell positions running. Skipping.
Symbol: ABNB.NAS
Action: Buy
⚠️ Failed to modify Buy position 3304860529. Error: Request returned None.
Symbol: ABNB.NAS
Action: Sell
⚠️ No Sell positions running. Skipping.
Symbol: ABT.NYSE
Action: Buy
⚠️ Failed to modify Buy position 3302221724. Error: AutoTrading disabled by client
Symbol: ABT.NYSE
Action: Sell
⚠️ No Sell positions running. Skipping.
Symbol: ACN.NYSE
Action: Buy
⚠️ Failed to modif

In [21]:
import MetaTrader5 as mt5
import pandas as pd

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# === Fetch Open Positions === #
positions = mt5.positions_get()
if positions is None or len(positions) == 0:
    print("No running trades found. Exiting.")
    mt5.shutdown()
    quit()

# === Retrieve Unique Symbols from Positions === #
symbols = sorted(list(set(pos.symbol for pos in positions)))

# === Create DataFrame for Symbols and Decimals === #
symbol_decimals = []

for symbol in symbols:
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info:
        symbol_decimals.append({
            "Symbol": symbol,
            "Decimals": symbol_info.digits
        })

# === Convert to DataFrame and Display === #
df_decimals = pd.DataFrame(symbol_decimals)
print(df_decimals)

# === Shutdown MT5 Connection === #
mt5.shutdown()


        Symbol  Decimals
0     AAPL.NAS         2
1      AAVEUSD         2
2    ABBV.NYSE         2
3     ABNB.NAS         2
4     ABT.NYSE         2
..         ...       ...
158     XBRUSD         2
159     XNGUSD         3
160   XOM.NYSE         2
161     XTIUSD         2
162     ZECUSD         2

[163 rows x 2 columns]


True

In [24]:
df_decimals.loc[(df_decimals["Symbol"] == "AUDCAD")]

,Symbol,Decimals
69,AUDCAD,5
